In [22]:
import requests
import pandas as pd
import numpy as np

EMAIL = 'gil@ficc.ai'
PASSWORD = 'aDwYwwL6'

def ficc_prices(cusips_quantities_tradetypes):
    url = 'https://api.ficc.ai/api/batchpricing'
    data = {'username': EMAIL, 
            'password': PASSWORD}
    cusip_list = [item['CUSIP'] for item in cusips_quantities_tradetypes]
    quantity_list = [item['Quantity'] for item in cusips_quantities_tradetypes]
    trade_type_list = [item['Trade Type'] for item in cusips_quantities_tradetypes]
    data['cusipList'] = cusip_list
    data['quantityList'] = quantity_list
    data['tradeTypeList'] = trade_type_list
    resp = requests.post(url, data=data)

    if resp.ok:
        try:
            return pd.read_json(resp.json())
        except Exception:
            pass
    print('batch error')

# Explanation of Current Condition Logic:
# For trades labeled as Bid, a negative price delta (meaning the user paid less than the API price) should be considered good.
# For trades labeled as Offered, a positive price delta (meaning the user sold at a higher price than the API price) should also be considered good.

def standardize_trade_type(trade_type):
    if 'Offered Side' in trade_type:
        return 'S'  # Sale to customer
    elif 'Bid Side' in trade_type:
        return 'P'  # Purchase from customer
    elif 'Inter-Dealer' in trade_type:
        return 'D'  # Dealer trades
    return trade_type  # Fallback, should align or highlight an issue

def evaluate_trades(trades):
    ficc_price_data = ficc_prices(trades)

    if not ficc_price_data.empty:
        user_data = pd.DataFrame(trades)
        user_data.rename(columns={'CUSIP': 'cusip', 'Quantity': 'quantity', 'Trade Type': 'trade_type', 'User Price': 'user_price'}, inplace=True)
        
        # Adjust user_data quantities to match API data representation
        user_data['quantity'] = user_data['quantity'] * 1000  # Convert thousands to full units

        # Standardize trade_type if necessary
        user_data['trade_type'] = user_data['trade_type'].apply(standardize_trade_type)

        # Merge the API response with the user data
        merged_data = ficc_price_data.merge(user_data, on=['cusip', 'quantity', 'trade_type'], how='left')

        # Calculate price differences considering the type of trade
        merged_data['Adjusted Price Delta'] = merged_data.apply(
            lambda row: row['price'] - row['user_price'] if row['trade_type'] in ['S', 'D'] else row['user_price'] - row['price'], axis=1
        )

        # Define the ratings based on adjusted price delta
        conditions = [
            (merged_data['Adjusted Price Delta'] > 0.06),
            (merged_data['Adjusted Price Delta'] > 0.03) & (merged_data['Adjusted Price Delta'] <= 0.06),
            (merged_data['Adjusted Price Delta'] > 0) & (merged_data['Adjusted Price Delta'] <= 0.03),
            (merged_data['Adjusted Price Delta'] <= 0)
        ]
        choices = ['Excellent', 'Good', 'Fair', 'Poor']
        merged_data['Rating'] = np.select(conditions, choices, default='Poor')

        return merged_data[['cusip', 'quantity', 'trade_type', 'price', 'user_price', 'Adjusted Price Delta', 'Rating']]
    else:
        print("No data to evaluate")
        return None

# Define your trades input correctly
trades = [
    {'CUSIP': '812627GN1', 'Quantity': 20, 'Trade Type': 'S', 'User Price': 104.858},
    {'CUSIP': '735389J33', 'Quantity': 20, 'Trade Type': 'D', 'User Price': 102.239},
    # Include all other trades as needed
]

results = evaluate_trades(trades)
if results is not None:
    print(results)
else:
    print("Failed to get results.")


# Ensure you re-test with this approach and check the outputs from the print statements


# Sample trades data including CUSIP, quantity, trade type, and user-reported price
trades = [
    {'CUSIP': '812627GN1', 'Quantity': 20, 'Trade Type': 'S', 'User Price': 104.858},
    {'CUSIP': '812627GN1', 'Quantity': 20, 'Trade Type': 'S', 'User Price': 104.858},
    {'CUSIP': '735389J33', 'Quantity': 20, 'Trade Type': 'D', 'User Price': 102.239},
    {'CUSIP': '735389J33', 'Quantity': 20, 'Trade Type': 'D', 'User Price': 102.166},
    {'CUSIP': '93978HRU1', 'Quantity': 40, 'Trade Type': 'P', 'User Price': 100.134},
    {'CUSIP': '812631MP1', 'Quantity': 35, 'Trade Type': 'D', 'User Price': 103.457},
    {'CUSIP': '812627FC6', 'Quantity': 30, 'Trade Type': 'P', 'User Price': 100.133},
    {'CUSIP': '735389YF9', 'Quantity': 15, 'Trade Type': 'D', 'User Price': 102.213},
    {'CUSIP': '735389J33', 'Quantity': 25, 'Trade Type': 'D', 'User Price': 102.249},
    {'CUSIP': '93978HMB8', 'Quantity': 10, 'Trade Type': 'D', 'User Price': 100.679},
    {'CUSIP': '812643LU6', 'Quantity': 10, 'Trade Type': 'P', 'User Price': 99.266}
]

# Call the evaluate_trades function with the sample trades
results = evaluate_trades(trades)

results


       cusip  quantity    trade_type    price  user_price  \
0  812627GN1     20000  Offered Side  103.935         NaN   
1  735389J33     20000  Inter-Dealer  102.279         NaN   

   Adjusted Price Delta Rating  
0                   NaN   Poor  
1                   NaN   Poor  


,cusip,quantity,trade_type,price,user_price,Adjusted Price Delta,Rating
0,812627GN1,20000,Offered Side,103.935,NaN,NaN,Poor
1,812627GN1,20000,Offered Side,103.935,NaN,NaN,Poor
2,735389J33,20000,Inter-Dealer,102.279,NaN,NaN,Poor
3,735389J33,20000,Inter-Dealer,102.279,NaN,NaN,Poor
4,93978HRU1,40000,Bid Side,100.177,NaN,NaN,Poor
5,812631MP1,35000,Inter-Dealer,103.232,NaN,NaN,Poor
6,812627FC6,30000,Bid Side,100.081,NaN,NaN,Poor
7,735389YF9,15000,Inter-Dealer,102.140,NaN,NaN,Poor
8,735389J33,25000,Inter-Dealer,102.292,NaN,NaN,Poor
9,93978HMB8,10000,Inter-Dealer,101.086,NaN,NaN,Poor
